# Profiling

cprofiler (standard module)

line_profiler

Intel Advisor 2017 beta

Before you start to optimize a program, you should generate a profile. There are several profiling tools,e.g.,  cprofile, which comes with Python and is always available. It measures performance on a function call level. We'll also look at line_profiler. Let's look at an example.

The following functions implement a simple n-body simulation using a long range potential. This could be part of, e.g., an astrophysical simulation, a simulation of a many-electron system, or a molecular dynamics simulation.


# Profiling a simple MD code

pair_force()

force()

calculate_all_forces()

step()

propagate_all_variables()

In [1]:
#%%writefile md.py
import numpy as np

def pair_force(x0, y0, z0, m0, x1, y1, z1, m1):
    """Calculate the force on p0 due to p1.
    
    Parameters
    ----------
    x0: float
        x-coordinate of the p0
    y0: float
        y-coordinate of the p0
    z0: float
        z-coordinate of the p0
    m0: float
        mass of the p0
    x1: float
        x-coordinate of the p1
    y1: float
        y-coordinate of the p1
    z1: float
        z-coordinate of the p1
    m1: float
        mass of the p1
    
    Returns
    -------
    f: ndarray
        force on p0 due to p1
        
    """
    r2 = (x1 - x0) ** 2 + (y1 - y0) ** 2 + (z1 - z0) ** 2
    f = m0 * m1 * np.array([(x1 - x0), (y1 - y0), (z1 - z0)]) * r2 ** (-1.5) if r2 else np.zeros(3)
    return f

def force(x0, y0, z0, m0, x, y, z, m):
    """Calculates the force on the particle at (x0, y0, z0) due to all other particles.
    
    Parameters
    ----------
    x0: float
        x-coordinate of the particle
    y0: float
        y-coordinate of the particle
    z0: float
        z-coordinate of the particle
    m0: float
        mass of the particle
    x: ndarray
        x-coordinates of all particles
    y: ndarray
        y-coordinates of all particles
    z: ndarray
        z-coordinates of all particles
    m: ndarray
        masses of all particles.
    
    Returns
    -------
    f: ndarray
        force on particle with mass m0 at (x0, y0, z0)
    """
    f = np.zeros(3)
    for x1, y1, z1, m1 in zip(x, y, z, m):
        f += pair_force(x0, y0, z0, m0, x1, y1, z1, m1)
    return f

def calculate_all_forces(x, y, z, m):
    """Calculates the force on each particle p due to all other particles.
    
    Parameters
    ----------
    x: ndarray
        x-coordinates of all particles
    y: ndarray
        y-coordinates of all particles
    z: ndarray
        z-coordinates of all particles
    m: ndarray
        masses of all particles.
    
    Returns
    -------
    f: ndarray
        force on each particle due to all other particles.
    """
    return np.array([force(x[i], y[i], z[i], m[i], x, y, z, m) for i in range(n)])

def step(x, y, z, vx, vy, vz, m, f, dt):
    """Propagate the position and velocities.
    
    Starting from the current positions, velocities, and forces, propogate positions
    and velocities by one time step of lenght dt.
    
    .. note:: This algorithm should not be used for real simulations!
    
    Parameters
    ----------
    x: ndarray
        x-coordinates of all particles
    y: ndarray
        y-coordinates of all particles
    z: ndarray
        z-coordinates of all particles
    vx: ndarray
        x-component of the velocity of all particles
    vy: ndarray
        y-component of the velocity of all particles
    vz: ndarray
        z-component of the velocity of all particles
    m: ndarray
        masses of all particles.
    f: ndarray
        forces on particles
    dt: float
        time step
    
    Returns
    -------
    x, y, z, vx, vy, vz at t + dt
    """
    xn = x + vx * dt + 0.5 * f[0] / m * dt * dt
    yn = y + vy * dt + 0.5 * f[1] / m * dt * dt
    zn = y + vz * dt + 0.5 * f[2] / m * dt * dt
    vxn = vx + f[0] / m * dt
    vyn = vy + f[1] / m * dt
    vzn = vz + f[2] / m * dt
    return xn, yn, zn, vxn, vyn, vzn

def propagate_all_variables(x, y, z, vx, vy, vz, m, f, dt):
    for i in range(n):
        x[i], y[i], z[i], vx[i], vy[i], vz[i] = step(x[i], y[i], z[i], vx[i], vy[i], vz[i], m[i], f[i], dt)

## The main program

## Initialization

In [2]:
# 1000 particles
n = 1000
# time step of 0.01
dt = 0.01

# Initialize coordinates and velocities to random values.
x = np.random.random(n)
y = np.random.random(n)
z = np.random.random(n)
vx = np.zeros_like(x)
vy = np.zeros_like(x)
vz = np.zeros_like(x)
m = np.ones_like(x)

### The algorithm

There are basically two steps to this algorithm:

1. Calculate the forces on all particles
2. Propagate all variables for a time step
3. Continue at 1. for nstep steps

In [4]:
%%timeit nsteps = 2
for i in range(nsteps):
    f = calculate_all_forces(x, y, z, m)
    propagate_all_variables(x, y, z, vx, vy, vz, m, f, dt)

1 loop, best of 3: 16.2 s per loop


This took quite some time. Let's measure how long it takes. Add a %%timeit statement just before nsteps (same line).

## The base line

In [6]:
%%timeit nsteps = 1
for i in range(nsteps):
    f = calculate_all_forces(x, y, z, m)
    propagate_all_variables(x, y, z, vx, vy, vz, m, f, dt)

1 loop, best of 3: 7.75 s per loop


OK, that's our base line. Next, we want to know where all this time is spent. I mentioned the cprofile module at the beginning. IPython has a magic for that called %%prun. Use it in front of the loop this time.

## Profiling with %%prun

In [22]:
%%prun -r nsteps=1
for i in range(nsteps):
    f = calculate_all_forces(x, y, z, m)
    propagate_all_variables(x, y, z, vx, vy, vz, m, f, dt)

    2003007 function calls in 8.561 seconds

    Ordered by: internal time

    ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    1000000  5.209    0.000    7.073    0.000 <ipython-input-3-66a2b576efa3>:4(pair_force)
    999001   1.863    0.000    1.863    0.000 {built-in method numpy.core.multiarray.array}
     1000    1.480    0.001    8.555    0.009 <ipython-input-3-66a2b576efa3>:36(force)
     2000    0.003    0.000    0.003    0.000 {built-in method numpy.core.multiarray.zeros}
     1000    0.003    0.000    0.003    0.000 <ipython-input-3-66a2b576efa3>:89(step)
        1    0.001    0.001    0.004    0.004 <ipython-input-3-66a2b576efa3>:130(propagate_all_variables)
        1    0.001    0.001    8.557    8.557 <ipython-input-3-66a2b576efa3>:87(<listcomp>)
        1    0.000    0.000    8.561    8.561 {built-in method builtins.exec}
        1    0.000    0.000    8.557    8.557 <ipython-input-3-66a2b576efa3>:68(calculate_all_forces)
        1    0.000    0.000    8.561    8.561 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

The overhead shouldn't be too bad. I got about 10%. Most of the time (about 80%) is spent in pair_force. And 20% of that time is spent on np.array>

## Line by line profiling

Unfortunately, this is a rather coarse grained profile. We don't know which part is the expensive part of this calculation and what we can do about it.

We can get some more information using line_profiler, but we can only do this on the command line. So let's switch to a terminal.

scpy@zam1068: kernprof -v -l md.py                                                                                                 
Wrote profile results to md.py.lprof                                                                                               
Timer unit: 1e-06 s                                                                                                                
                                                                                                                                   
Total time: 9.86691 s                                                                                                              
File: md.py                                                                                                                        
Function: pair_force at line 3                                                                                                     
                                                                                                                                   
    Line #      Hits         Time  Per Hit   % Time  Line Contents
    ==============================================================                                                       
        32   1000000      2269473      2.3     23.0      r2 = (x1 - x0) ** 2 + (y1 - y0) ** 2 + (z1 - z0) ** 2
        33   1000000      6864624      6.9     69.6      f = m0 * m1 * np.array([(x1 - x0), (y1 - y0), (z1 - z0)])  
                                                         * r2 **(-1.5) if r2 else np.zeros(3)
        34   1000000       732809      0.7      7.4      return f                                                                      


## Timing individual operations

Calculating r2 takes 2.3 mus per call. Let's use %timeit to see how much time each operation takes.

In [16]:
%%timeit x0 = x[0]; y0=y[0]; z0 = z[0]; x1 = x[1]; y1=y[1]; z1 = z[1];
(x1 - x0)


The slowest run took 76.60 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 64.8 ns per loop


In [17]:
%%timeit x0 = x[0]; y0=y[0]; z0 = z[0]; x1 = x[1]; y1=y[1]; z1 = z[1];
(x1 - x0) ** 2

The slowest run took 29.33 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 295 ns per loop


In [18]:
%%timeit x0 = x[0]; y0=y[0]; z0 = z[0]; x1 = x[1]; y1=y[1]; z1 = z[1];
(x1 - x0) * (x1 - x0)

The slowest run took 19.39 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 192 ns per loop


In [15]:
%%timeit x0 = x[0]; y0=y[0]; z0 = z[0]; x1 = x[1]; y1=y[1]; z1 = z[1];
dx = (x1 - x0)
dx * dx

The slowest run took 26.89 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 125 ns per loop


We can now change the code so it calculates dx, dy, and dz first and then uses them later in the calculation. We can also use numba to speed up the simulation.

## Exercise: Time the other operations and optimize the code

In [16]:
%%writefile md.py
import numpy as np
from numba import jit


def pair_force(x0, y0, z0, m0, x1, y1, z1, m1):
    """Calculate the force on p0 due to p1.
    
    Parameters
    ----------
    x0: float
        x-coordinate of the p0
    y0: float
        y-coordinate of the p0
    z0: float
        z-coordinate of the p0
    m0: float
        mass of the p0
    x1: float
        x-coordinate of the p1
    y1: float
        y-coordinate of the p1
    z1: float
        z-coordinate of the p1
    m1: float
        mass of the p1
    
    Returns
    -------
    f: ndarray
        force on p0 due to p1
        
    """
    
    # r2 = (x1 - x0) ** 2 + (y1 - y0) ** 2 + (z1 - z0) ** 2
    xx = (x1 - x0)
    yy = (y1 - y0)
    zz = (z1 - z0)
    r2 = xx * xx + yy * yy + zz * zz
    f = m0 * m1 * np.array([(x1 - x0), (y1 - y0), (z1 - z0)]) * r2 ** (-1.5) if r2 else np.zeros(3)
    return f


def force(x0, y0, z0, m0, x, y, z, m):
    """Calculates the force on the particle at (x0, y0, z0) due to all other particles.
    
    Parameters
    ----------
    x0: float
        x-coordinate of the particle
    y0: float
        y-coordinate of the particle
    z0: float
        z-coordinate of the particle
    m0: float
        mass of the particle
    x: ndarray
        x-coordinates of all particles
    y: ndarray
        y-coordinates of all particles
    z: ndarray
        z-coordinates of all particles
    m: ndarray
        masses of all particles.
    
    Returns
    -------
    f: ndarray
        force on particle with mass m0 at (x0, y0, z0)
    """
    f = np.zeros(3)
    for x1, y1, z1, m1 in zip(x, y, z, m):
        f += pair_force(x0, y0, z0, m0, x1, y1, z1, m1)
    return f


def calculate_all_forces(x, y, z, m):
    """Calculates the force on each particle p due to all other particles.
    
    Parameters
    ----------
    x: ndarray
        x-coordinates of all particles
    y: ndarray
        y-coordinates of all particles
    z: ndarray
        z-coordinates of all particles
    m: ndarray
        masses of all particles.
    
    Returns
    -------
    f: ndarray
        force on each particle due to all other particles.
    """
    return np.array([force(x[i], y[i], z[i], m[i], x, y, z, m) for i in range(n)])


def step(x, y, z, vx, vy, vz, m, f, dt):
    """Propagate the position and velocities.
    
    Starting from the current positions, velocities, and forces, propogate positions
    and velocities by one time step of lenght dt.
    
    .. note:: This algorithm should not be used for real simulations!
    
    Parameters
    ----------
    x: ndarray
        x-coordinates of all particles
    y: ndarray
        y-coordinates of all particles
    z: ndarray
        z-coordinates of all particles
    vx: ndarray
        x-component of the velocity of all particles
    vy: ndarray
        y-component of the velocity of all particles
    vz: ndarray
        z-component of the velocity of all particles
    m: ndarray
        masses of all particles.
    f: ndarray
        forces on particles
    dt: float
        time step
    
    Returns
    -------
    x, y, z, vx, vy, vz at t + dt
    """
    xn = x + vx * dt + 0.5 * f[0] / m * dt * dt
    yn = y + vy * dt + 0.5 * f[1] / m * dt * dt
    zn = y + vz * dt + 0.5 * f[2] / m * dt * dt
    vxn = vx + f[0] / m * dt
    vyn = vy + f[1] / m * dt
    vzn = vz + f[2] / m * dt
    return xn, yn, zn, vxn, vyn, vzn

def propagate_all_variables(x, y, z, vx, vy, vz, m, f, dt):
    for i in range(n):
        x[i], y[i], z[i], vx[i], vy[i], vz[i] = step(x[i], y[i], z[i], vx[i], vy[i], vz[i], m[i], f[i], dt)
        

if __name__ == '__main__':
    
    # 1000 particles
    n = 1000
    # time step of 0.01
    dt = 0.01

    # Initialize coordinates and velocities to random values.
    x = np.random.random(n)
    y = np.random.random(n)
    z = np.random.random(n)
    vx = np.zeros_like(x)
    vy = np.zeros_like(x)
    vz = np.zeros_like(x)
    m = np.ones_like(x)

    nsteps = 1
    for i in range(nsteps):
        f = calculate_all_forces(x, y, z, m)
        propagate_all_variables(x, y, z, vx, vy, vz, m, f, dt)

Overwriting md.py


In [7]:
%%timeit nsteps = 1
for i in range(nsteps):
    f = calculate_all_forces(x, y, z, m)
    propagate_all_variables(x, y, z, vx, vy, vz, m, f, dt)

1 loop, best of 3: 7.85 s per loop


In [18]:
%%timeit nsteps = 1
for i in range(nsteps):
    f = calculate_all_forces(x, y, z, m)
    propagate_all_variables(x, y, z, vx, vy, vz, m, f, dt)

1 loop, best of 3: 7.47 s per loop


In [19]:
%%prun -r nsteps = 1
for i in range(nsteps):
    f = calculate_all_forces(x, y, z, m)
    propagate_all_variables(x, y, z, vx, vy, vz, m, f, dt)

In [ ]:
%%